In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from utils import read_pickle
from tqdm.notebook import tqdm

In [313]:
path = '/Users/diegofiori/Desktop/epfl/master_thesis/results/'
sim_code = 'qs'
density = read_pickle(path+f'physical_features_end_{sim_code}.pickle')
slices = read_pickle(path+f'slices_top_features_end_{sim_code}.pickle')

In [314]:
density = np.concatenate(density)

In [315]:
density.shape

(268,)

In [316]:
slices.shape

(5, 7680, 11134)

In [317]:
df = pd.DataFrame(data=slices[0])

In [318]:
df.shape

(7680, 11134)

In [319]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11124,11125,11126,11127,11128,11129,11130,11131,11132,11133
0,2.531955,2.979229,0.997591,0.873178,1.919073,2.113158,1.011299,0.918527,0.497564,0.381366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.533453,2.945948,0.997570,0.872496,1.916555,2.097693,1.011165,0.918383,0.497544,0.380756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.552748,3.018794,0.997560,0.871683,1.926528,2.146265,1.011264,0.918488,0.497534,0.380017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.577988,3.000581,0.997562,0.870987,1.943366,2.138998,1.011615,0.918619,0.497535,0.379380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.563686,2.963647,0.997585,0.870331,1.936030,2.133089,1.011507,0.918777,0.497558,0.378784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [320]:
from pipeline_builder import get_pipeline_index

In [321]:
dict_transf = get_pipeline_index()
dict_transf

{'persistence_entropy': I[0, 2),
 'amplitude_bottleneck_': I[2, 4),
 'amplitude_wasserstein_1': I[4, 6),
 'amplitude_wasserstein_2': I[6, 8),
 'amplitude_landscape_1_1_100': I[8, 10),
 'amplitude_landscape_1_2_100': I[10, 12),
 'amplitude_landscape_2_1_100': I[12, 14),
 'amplitude_landscape_2_2_100': I[14, 16),
 'amplitude_betti_1_100': I[16, 18),
 'amplitude_betti_2_100': I[18, 20),
 'amplitude_heat_1_1.6_100': I[20, 22),
 'amplitude_heat_1_3.2_100': I[22, 24),
 'amplitude_heat_2_1.6_100': I[24, 26),
 'amplitude_heat_2_3.2_100': I[26, 28),
 'derivative_bottleneck_': I[28, 30),
 'derivative_wasserstein_1': I[30, 32),
 'derivative_wasserstein_2': I[32, 34),
 'heat_kernel_1.6': I[34, 5034),
 'heat_kernel_3.2': I[5034, 10034),
 'betti_curve': I[10034, 10134),
 'pers_landscape': I[10134, 11134)}

## Selecting features based on RandomForest

We firstly defined the regression problem.

In [322]:
import h5py

In [323]:
from scipy.io import loadmat
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code}.mat')
y = x['k_quantities']

In [324]:
y = np.concatenate([y[i, :, :] for i in range(y.shape[0])])

In [325]:
y.shape

(8000, 2)

In [326]:
y = y[:len(df)]

In [327]:
non_zero_ind = np.any(y>0, axis=1)

In [328]:
y1 = y[non_zero_ind, 0]
y2 = y[non_zero_ind, 1]

In [329]:
X = df.values[non_zero_ind]

In [330]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [331]:
from scipy.stats import pearsonr
def compute_correlation_with_old_features(target_feat, old_features_idx, X):
    corr = [pearsonr(target_feat, X[:, old_idx])[0] for old_idx in old_features_idx]
    return np.abs(corr)

In [332]:
# remove some features which are higly correlated
X = np.nan_to_num(X)
columns_idx = []
columns_tuple_id = [(i, j) for i in range(slices.shape[0]) for j in range(slices.shape[2])]
equi_class = {}
class_id = 0
for i in tqdm(range(X.shape[1])):  
    feat_mat = X[:, i]
    if columns_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, columns_idx, X)]
        if np.max(corrs) < 0.95:
            columns_idx.append(i)
            equi_class[class_id] = [columns_tuple_id[i]]
            class_id += 1
        else:
            equi_class[np.argmin(corrs)].append(columns_tuple_id[i])
    else:
        columns_idx.append(i)
        equi_class[class_id] = [columns_tuple_id[i]]
        class_id += 1
X = X[:, np.array(columns_idx)]

/Users/diegofiori/anaconda3/envs/giotto_tda_env/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/diegofiori/anaconda3/envs/giotto_tda_env/lib/python3.8/site-packages/scipy/stats/stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [333]:
equi_class = {equi_class[i][0]: equi_class[i] for i in equi_class.keys()}

In [334]:
X.shape

(7680, 1345)

In [335]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2)

In [336]:
model1 = RandomForestRegressor(n_estimators=2000)
model1.fit(X_train, y1_train)
model2 = RandomForestRegressor(n_estimators=2000)
model2.fit(X_train, y2_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [337]:
score_1 = model1.score(X_test, y1_test)
score_2 = model2.score(X_test, y2_test)
total_score = score_1 + score_2

In [338]:
score_2

0.9792675981196219

In [339]:
score_1

0.9556233540907576

In [340]:
total_score

1.9348909522103797

In [341]:
model1.predict(X_test[19:20])

array([0.02521032])

In [342]:
y1_test[19]

0.024545156151915628

In [343]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y1_test, model1.predict(X_test))

0.0004783513754910345

In [344]:
print('relative error')
mean_absolute_error(y1_test, model1.predict(X_test))/np.mean(y1_test)

relative error


0.017058091036685718

In [345]:
feature_importance = score_1/total_score*model1.feature_importances_ + score_2/total_score*model2.feature_importances_


In [346]:
# group the features (we want to keep all the spatial dimensions)
n_features = slices.shape[2]
#feature_importance_temp = [feature_importance[i:i+n_features] for i in range(0, len(feature_importance), n_features)]
#feature_importance_temp = [np.mean(feature_importance_temp[i:i+80], axis=0) 
#                           for i in range(0, len(feature_importance_temp), 80)]

In [347]:
feature_importance_idx = [j for j in range(n_features)]
feature_importance_idx = [feature_importance_idx[i] for i in range(len(feature_importance_idx)) if i in columns_idx]
feature_importance_idx_sort = sorted(feature_importance_idx, reverse=True,
                                key=lambda x: feature_importance[feature_importance_idx.index(x)])

In [348]:
important_features_idx = []
threshold = 0.85
minimum_value = np.max(feature_importance)/100
for i, idx in enumerate(tqdm(feature_importance_idx_sort)):
    feat_value = feature_importance[feature_importance_idx.index(idx)]
    if feat_value < minimum_value:
        break
    feat_mat = X[:, feature_importance_idx.index(idx)]
    if important_features_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, important_features_idx, X)]
        if np.max(corrs) < threshold:
            important_features_idx.append(i)
        else:
            list_temp = equi_class.pop((0, idx))
            equi_class[(0, feature_importance_idx_sort[important_features_idx[np.argmax(corrs)]])] += list_temp
    else:
        important_features_idx.append(i)
important_features = [feature_importance_idx_sort[i] for i in important_features_idx]

In [349]:
len(equi_class)

1332

In [350]:
important_equi_class = {(0, key): equi_class[(0, key)] for key in important_features}

In [351]:
len(important_equi_class)

55

In [352]:
from utils import write_pickle
write_pickle(path=path+f'selected_index_{sim_code}_single_field.pickle', array=important_features)

In [353]:
write_pickle(path=path+f'selected_equi_class_{sim_code}_single_field.pickle', array=important_equi_class)

In [312]:
len(important_features)

101

In [52]:
important_features = [(0, i) for i in important_features]

In [53]:
important_equi_class = {key: [(0, i) for i in important_equi_class[key]]
                        for key in important_equi_class.keys()}

In [54]:
important_features_all = read_pickle(path+'selected_index_cs.pickle')

In [55]:
intersection = set(important_features_all).intersection(set(important_features))

In [56]:
intersection

{(0, 4384),
 (0, 4385),
 (0, 10185),
 (0, 10186),
 (0, 10236),
 (0, 10637),
 (0, 10638),
 (0, 10639),
 (0, 10688),
 (0, 10689),
 (0, 10690),
 (0, 10691),
 (0, 10692),
 (0, 10699),
 (0, 10717),
 (0, 10842)}

In [57]:
intersection_1 = set(important_features_all).intersection(set([(1, x[1]) for x in important_features]))
intersection_1

{(1, 6),
 (1, 2384),
 (1, 10185),
 (1, 10186),
 (1, 10188),
 (1, 10235),
 (1, 10236),
 (1, 10637),
 (1, 10687),
 (1, 10688),
 (1, 10692),
 (1, 10737),
 (1, 10787)}

In [58]:
intersection_2 = set(important_features_all).intersection(set([(2, x[1]) for x in important_features]))
intersection_2

{(2, 0),
 (2, 6),
 (2, 10186),
 (2, 10187),
 (2, 10188),
 (2, 10193),
 (2, 10236),
 (2, 10239),
 (2, 10286),
 (2, 10638),
 (2, 10639),
 (2, 10692),
 (2, 10695),
 (2, 10696),
 (2, 10698),
 (2, 10699),
 (2, 10741),
 (2, 10745),
 (2, 10793)}

In [59]:
len(intersection.union(intersection_1).union(intersection_2))/len(important_features_all)

0.41739130434782606

In [60]:
len(important_features)/len(important_features_all)

0.6695652173913044

In [61]:
0.443/0.829

0.5343787696019301

## Compare equi_classes

In [65]:
important_equi_class_all = read_pickle(path+f'selected_equi_class_{sim_code}.pickle')

In [90]:
important_equi_class = {key: [x[1] for x in important_equi_class[key]]
                        for key in important_equi_class.keys()}

In [87]:
def intersect_equi_classes(equi_1, equi_2):
    common_eq_classes = {}
    class_idx = 0
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) > 0:
                common_eq_classes[class_idx] = list(set_res)
                class_idx += 1
    return common_eq_classes

In [103]:
def union_equi_classes(equi_1, equi_2):
    union_eq_classes = {i: equi_2[key] for i, key in enumerate(equi_2.keys())}
    class_idx = len(union_eq_classes)
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) == 0:
                union_eq_classes[class_idx] = equi_1[key_1]
                class_idx += 1
                break
    return union_eq_classes

In [93]:
intersection_equi_class = intersect_equi_classes(important_equi_class_all, important_equi_class)

In [104]:
union_class = union_equi_classes(important_equi_class_all, important_equi_class)

In [105]:
len(union_class)

192

In [106]:
len(intersection_equi_class)/len(union_class)

0.11458333333333333

## Random Forest using only the important features for both the single field and the total problem

Case 1: single_simulation

In [53]:
X1 = np.concatenate([slices[i, :, j].reshape((-1, 1)) for (i, j) in important_features_all], axis=1)

In [54]:
X2 = np.concatenate([slices[i, :, j].reshape((-1, 1)) for (i, j) in important_features], axis=1)

In [57]:
print(X1.shape, X2.shape, y1.shape, y2.shape)

(7680, 88) (7680, 73) (7680,) (7680,)


In [58]:
X1_train, X1_test, X2_train, X2_test, y1_train, y1_test, y2_train, y2_test = \
    train_test_split(X1, X2, y1, y2, test_size=0.2)

In [84]:
model_11 = RandomForestRegressor(n_estimators=100)
model_12 = RandomForestRegressor(n_estimators=100)
model_21 = RandomForestRegressor(n_estimators=100)
model_22 = RandomForestRegressor(n_estimators=100)

In [85]:
model_11.fit(X1_train, y1_train)
model_12.fit(X1_train, y2_train)
model_21.fit(X2_train, y1_train)
model_22.fit(X2_train, y2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [86]:
score_11 = mean_absolute_error(y1_test, model_11.predict(X1_test))
score_12 = mean_absolute_error(y2_test, model_12.predict(X1_test))
score_21 = mean_absolute_error(y1_test, model_21.predict(X2_test))
score_22 = mean_absolute_error(y2_test, model_22.predict(X2_test))

In [87]:
print(score_11, score_12, score_21, score_22)

0.000384319467149441 0.0007430684626557733 0.0004889102142923754 0.0010202162621454523


In [89]:
print(np.mean(y1), y2.mean())

0.03073030972208013 0.08123853839464416


Case 2: use another simulation as test

In [63]:
sim_code_test = 'sf'
slices_test =  read_pickle(path+f'slices_top_features_end_{sim_code_test}.pickle')

In [64]:
model_11 = RandomForestRegressor(n_estimators=100)
model_12 = RandomForestRegressor(n_estimators=100)
model_21 = RandomForestRegressor(n_estimators=100)
model_22 = RandomForestRegressor(n_estimators=100)

In [65]:
model_11.fit(X1, y1)
model_12.fit(X1, y2)
model_21.fit(X2, y1)
model_22.fit(X2, y2)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [77]:
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code_test}.mat')
y_new = x['k_quantities']

In [68]:
X1_new = np.concatenate([slices_test[i, :, j].reshape(-1, 1) for i, j in important_features_all], axis=1)
X2_new = np.concatenate([slices_test[i, :, j].reshape(-1, 1) for i, j in important_features], axis=1)

In [78]:
y_new = np.concatenate([y_new[i, :, :] for i in range(y_new.shape[0])])
y_new = y_new[:len(X1_new)]
y1_new = y_new[:, 0]
y2_new = y_new[:, 1]

In [79]:
y1_new.shape

(7680,)

In [82]:
score_11 = mean_absolute_error(y1_new, model_11.predict(X1_new))
score_12 = mean_absolute_error(y2_new, model_12.predict(X1_new))
score_21 = mean_absolute_error(y1_new, model_21.predict(X2_new))
score_22 = mean_absolute_error(y2_new, model_22.predict(X2_new))

In [83]:
print(score_11, score_12, score_21, score_22)

0.009043744755882745 0.0038099001060633493 0.009355945201310075 0.00471834558049169


In [90]:
print(np.mean(y1_new), y2_new.mean())

0.039276038699720475 0.08282099771353216
